<a href="https://colab.research.google.com/github/Seoyoung0519/DataStructures_Assignment06/blob/main/SSU_%EA%B0%9C%EC%84%A4_%EA%B0%95%EC%A2%8C_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SSU 개설 강좌 크롤러**

성신여자대학교 수강신청 시스템의 “개설 강좌 조회” 결과를 한 번에 가져와 원하는 컬럼만 골라 .xlsx 로 저장해 주는 파이썬 스크립트이다.


##**1. 필수 패키지**

In [ ]:
pip install requests
pip install pandas
pip install openpyxl

## **2. AJAX 엔드포인트 파악**

1. 개설강좌 조회 브라우저 F12 → Network(Fetch/XHR) 탭

2. “과목 조회” 버튼(실제론 자동 호출) 직후에 발생하는
findBCRM02010MainListIn.do 요청 확인

3. Headers → Form Data 에 아래 키·값이 보임





In [ ]:
yy=2025
semCd=COMM063.10
orgClsfCd=COMM075.101
sbjMngCd=
objCrsCd=USSR001.10
…
charSbjAreaCd=


--> 이 파라미터 값을 그대로 스크립트의 payload 에 복사

## **3. 스크립트 설명**

**주요 포인트**
1. Payload

- DevTools → Network → Form Data에서 복사한 필터 값을 그대로 사용

2. JSON → DataFrame

- resp.json() → pd.DataFrame

3. 컬럼 선택

- df[cols_to_keep] 로 필요 열만 추출

4. 컬럼명 한글화

- rename() 으로 보기 좋게 변경

5. 엑셀 저장

- to_excel(..., engine="openpyxl")

In [ ]:
import requests
import pandas as pd

# — 환경 설정 —
URL = "https://sugang.sungshin.ac.kr/findBCRM02010Main.do"
payload = {
    "yy":"2025",                 # 학년도
    "semCd":"COMM063.10",        # 학기 코드
    "orgClsfCd":"COMM075.101",   # 단과대 코드
    "sbjMngCd":"",               # (빈 값 = 전체)
    "objCrsCd":"USSR001.10",     # 과정 구분 코드
    "dptMjrCd":"",               # 전공 코드 (빈 값 = 전체)
    "sbjNoNm":"",                # 과목명 필터 (빈 값 = 전체)
    "cpdivCd":"", "cmpCd":"",
    "sbjAreaCd":"", "charSbjAreaCd":""
}
session = requests.Session()
session.headers.update({
    "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8"
})

# 1) API 호출 & JSON 파싱
resp = session.post(URL, data=payload)
data = resp.json()  # 리스트 of dict

# 2) DataFrame 생성
df = pd.DataFrame(data)

# 3) 원본 컬럼 확인
print(df.columns.tolist())

# 4) 원하는 컬럼만 서브셋
cols_to_keep = [
    "opDptmjrNm","sbjDetCd","sbjNm","dvcls","cpdivNm",
    "sbjAreaNm","totLmtCnt","cdtExchgPsblYn","cdtHcnt",
    "tmtblKorDsc","roomKorDsc","profDsc","cmpCdNm",
    "lesnTypNm","rmkDsc"
]
df_subset = df[cols_to_keep]

# 5) 한글 컬럼명으로 변경
df_subset = df_subset.rename(columns={
    "opDptmjrNm":"개설학과전공","sbjDetCd":"학수번호",
    "sbjNm":"교과목명","dvcls":"분반","cpdivNm":"이수구분",
    "sbjAreaNm":"영역","totLmtCnt":"수강정원",
    "cdtExchgPsblYn":"학점교류","cdtHcnt":"학점/이론/실습",
    "tmtblKorDsc":"시간표","roomKorDsc":"강의실",
    "profDsc":"교수","cmpCdNm":"캠퍼스","lesnTypNm":"수업유형",
    "rmkDsc":"수강안내 및 지정내용"
})

# 6) 엑셀로 저장
output = "SSU_개설강좌_필요컬럼만.xlsx"
df_subset.to_excel(output, index=False, engine="openpyxl")
print(f"✅ 저장 완료: {output}")


## **4. SSW 25년 1학기 개설강좌 조회 xlsx 파일**